## Saving torch models

* design
* train
* save

##### imports

In [1]:
import torch
from torchvision import datasets, transforms
import helper
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

# Transformation
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

#### model design

In [2]:
# get the size of the input
sample_image,label = next(iter(testloader))
size_ = sample_image.shape
print(size_)

torch.Size([64, 1, 28, 28])


In [3]:
import torch.nn.functional as F
from torch import nn
from torch import optim


class Network(nn.Module):
    def __init__(self,input_shape,output_shape,number_of_hidden_layers,nodes_in_layers,dropouts):
        super().__init__()
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.number_of_hidden_layers = number_of_hidden_layers
        self.nodes_in_layers = nodes_in_layers
        self.dropout_percs = dropouts
        self.layers = nn.ModuleList()
        self.set_layers()
        
                         
                         
    def set_layers(self):
        self.layers.append(nn.Linear(input_shape[0],nodes_in_layers[0]))
        for k in range(number_of_hidden_layers):
            if k<self.number_of_hidden_layers-1:
                self.layers.append(nn.Linear(nodes_in_layers[k],nodes_in_layers[k+1]))
            else:
                self.layers.append(nn.Linear(nodes_in_layers[k],output_shape))
    
    def forward(self,x):
        x = x.view(x.shape[0], -1)# flatten the image
        m=0
        for k in range(self.number_of_hidden_layers):
            if k<self.number_of_hidden_layers-1:
                x = F.relu(self.layers[k](x))
            else:
                x = F.log_softmax(self.layers[k](x), dim=1)
            m+=1
        return x

def accuracy(y_hat_tensor,label_tensor):
    '''
    args:
        y_hat_tensor tensor: direct output of the model. 
        label_tensor tensor: actual labels of the given items
    returns:
        accuracy float
        accurate float: number of accurately labeled items
        total_samples float : number of samples investigated
    '''
    y_hat_tensor = torch.exp(y_hat_tensor)
    values, pred_labels = y_hat_tensor.max(1) # works like numpy argmax plus returns the values of the cells.
    accurate = sum(1 for a, b in zip(pred_labels.numpy(), label_tensor.numpy()) if a == b)
    total_samples = len(label_tensor)
    accuracy = accurate/total_samples
    return accuracy,accurate,total_samples

In [4]:
input_shape = [28*28]
output_shape = 10
number_of_hidden_layers = 3
nodes_in_layers = [128,64,32]
dropouts=[]
model = Network(input_shape,output_shape,number_of_hidden_layers,nodes_in_layers,dropouts)
print(model)

Network(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=10, bias=True)
  )
)


#### Loss function and optimiser

In [5]:
criterion = nn.NLLLoss() # Loss function Negative log likelyhood loss
optimizer = optim.Adam(model.parameters(), lr=0.01) #  learning rate 0.003

#### train

In [7]:
epochs = 3
epoch = 0
train_losses = []
test_losses = []
for e in range(epochs):
    running_loss = 0
    total_accurate = 0
    total_samples = 0
    for images, labels in trainloader:
        # Training pass
        output = model(images) # directly passes the images into forward method
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_train_accuracy,accurate,total_sample = accuracy(output,labels)
        running_loss += loss.item()
        total_accurate += accurate
        total_samples += total_sample
    else:
        with torch.no_grad():
            model.eval()
            test_loss = 0
            total_samples_test = 0
            total_accurate_test = 0
            for images, labels in testloader:
                output = model(images)
                test_loss += criterion(output, labels)
                batch_test_accuracy,accurate_test,total_sample_test = accuracy(output,labels)
                total_accurate_test += accurate_test
                total_samples_test += total_sample_test
        model.train()
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))
        print('''---------- epoch : {} -----------'''.format(epoch+1))
        print(''' Training Accuracy = {}  - Training Loss = {}'''.format(total_accurate/total_samples,running_loss/len(trainloader)))
        print(''' Test Accuracy = {}  - Test Loss = {}'''.format(total_accurate_test/total_samples_test,test_loss/len(testloader)))
        epoch += 1

---------- epoch : 1 -----------
 Training Accuracy = 0.8530666666666666  - Training Loss = 0.4147444701493422
 Test Accuracy = 0.8464  - Test Loss = 0.4446774125099182
---------- epoch : 2 -----------
 Training Accuracy = 0.8559833333333333  - Training Loss = 0.40338617709399793
 Test Accuracy = 0.8297  - Test Loss = 0.4986751079559326
---------- epoch : 3 -----------
 Training Accuracy = 0.8603333333333333  - Training Loss = 0.3987375376924777
 Test Accuracy = 0.8405  - Test Loss = 0.4593496024608612


#### save model

In [25]:
checkpoint = {
    'input_size' : input_shape[0],
    'output_size' : output_shape,
    'hidden_layers' : [layer.out_features for layer in model.layers[1:-1]],
    'state_dict' : model.state_dict()
    
}

torch.save(model.state_dict(),'..\models\\4_sample_model.pth')

In [26]:
print(model.state_dict().keys())

odict_keys(['layers.0.weight', 'layers.0.bias', 'layers.1.weight', 'layers.1.bias', 'layers.2.weight', 'layers.2.bias', 'layers.3.weight', 'layers.3.bias'])


#### save entire model

In [27]:
torch.save(model,'..\models\\entire_sample_model.pth')

c:\users\sismc\desktop\projects\pytorchstudy\pts\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
c:\users\sismc\desktop\projects\pytorchstudy\pts\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
c:\users\sismc\desktop\projects\pytorchstudy\pts\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
